### imports and config

In [11]:
%load_ext autoreload
%autoreload 2

import boto3
from langchain_community.chat_models import BedrockChat
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (ReActJsonSingleInputOutputParser, ReActSingleInputOutputParser)
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser

import sys 
# appending a path - run only once
# sys.path.append('/home/ubuntu/agi-cookbook/agents/patterns') 
print(sys.path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['/home/ubuntu/agi-cookbook/agents/patterns', '/opt/conda/envs/pytorch/lib/python310.zip', '/opt/conda/envs/pytorch/lib/python3.10', '/opt/conda/envs/pytorch/lib/python3.10/lib-dynload', '', '/opt/conda/envs/pytorch/lib/python3.10/site-packages']


### prompts

#### example from hub

In [2]:
from langchain import hub

prompt = hub.pull("hwchase17/xml-agent-convo")
# prompt.pretty_print()

#### simple-xml

In [3]:
%%writefile utils/xml_prompt.py
from typing import Sequence
from langchain.tools import BaseTool
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate

AGENT_PREFIX = """You are an AI assistant. You always stays on topic of the human input and does not diverge from it."""

AGENT_FORMAT_INSTRUCTIONS = """
You have access to the following tools:
<tools>
{tool_names}
</tools>

You have access to below tool descriptions:
<tool_descriptions>
{tool_descriptions}
</tool_descriptions>

Use the following format:
Question: the input question you must answer.
Thought: you should always think about what to do
Action: the action to take, should be one of tools mentioned within <tools> XML tags.
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

You have access to the previous conversation below, where H refers to the human and A refers to the assistant:
<chat_history>
{chat_history}
</chat_history>
"""

AGENT_QUESTION_PROMPT = """Remember to respond with your knowledge when the question does not correspond to any tool. 
Always append the string "Final Answer:" when returning the final answer.

Question: {question}"""

AGENT_SUFFIX = """Thought: {agent_scratchpad}"""

def create_agent_prompt(
        tools: Sequence[BaseTool],
        prefix: str = AGENT_PREFIX,
        # context: str = AGENT_CONTEXT,
        format_instructions: str = AGENT_FORMAT_INSTRUCTIONS,
        question_prompt: str = AGENT_QUESTION_PROMPT,
        suffix: str = AGENT_SUFFIX,
    ) -> PromptTemplate:
    
        human_prompt = PromptTemplate(
            input_variables=[
                "chat_history", 
                "question",
                "agent_scratchpad"
            ],
            partial_variables={
                "tool_names": ", ".join([tool.name for tool in tools]),
                "tool_descriptions": "\n".join(
                    [f"    {tool.name}: {tool.description}" for tool in tools]
                )
            },
            template='\n'.join(
                [
                    prefix,
                    format_instructions,
                    # context,
                    question_prompt,
                    suffix
                ]
            )
        )
        human_message_prompt = HumanMessagePromptTemplate(prompt=human_prompt)
        return ChatPromptTemplate.from_messages([human_message_prompt])

Overwriting utils/xml_prompt.py


In [ ]:
import re
from typing import List, Union
from langchain.agents import  AgentOutputParser
from langchain.schema import AgentAction, AgentFinish

class CustomOutputParser(AgentOutputParser):    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # print("llm_output",llm_output)
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()

#### xml-blob in action

In [12]:
# %%writefile utils/xml_prompt.py
# from typing import Sequence
# from langchain.tools import BaseTool
# from langchain.prompts import PromptTemplate, ChatPromptTemplate
# from langchain.prompts.chat import HumanMessagePromptTemplate

# AGENT_PREFIX = """You are an AI assistant. You always stays on topic of the human input and does not diverge from it."""

# AGENT_FORMAT_INSTRUCTIONS = """
# You have access to below tool descriptions:
# <tool_descriptions>
# {tool_descriptions}
# </tool_descriptions>

# The $XML_BLOB should only contain a SINGLE tool and MUST be formatted as markdown, do NOT return a list of multiple tools. Here is an example of a valid $XML_BLOB:

# ```xml
# <tool>$TOOL_NAME</tool>
# <tool_input>$INPUT</tool_input>
# ```
# Make sure to have the $INPUT in the right format for the tool you are using, and do not put variable names as input if you can find the right values.

# Use the following format:
# Question: the input question you must answer.
# Thought: you should always think about what to do
# Action: the tool in the $XML_BLOB should be one of [{tool_names}]
# ```xml
# $XML_BLOB
# ```
# Action Input: the input to the action
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can repeat N times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question

# You have access to the previous conversation below, where H refers to the human and A refers to the assistant:
# <chat_history>
# {chat_history}
# </chat_history>
# """

# AGENT_QUESTION_PROMPT = """Remember to respond with your knowledge when the question does not correspond to any tool. 
# Always append the string "Final Answer:" when returning the final answer.

# Question: {question}"""

# AGENT_SUFFIX = """Thought: {agent_scratchpad}"""

# def create_agent_prompt(
#         tools: Sequence[BaseTool],
#         prefix: str = AGENT_PREFIX,
#         # context: str = AGENT_CONTEXT,
#         format_instructions: str = AGENT_FORMAT_INSTRUCTIONS,
#         question_prompt: str = AGENT_QUESTION_PROMPT,
#         suffix: str = AGENT_SUFFIX,
#     ) -> PromptTemplate:
    
#         human_prompt = PromptTemplate(
#             input_variables=[
#                 "chat_history", 
#                 "question",
#                 "agent_scratchpad"
#             ],
#             partial_variables={
#                 "tool_names": ", ".join([tool.name for tool in tools]),
#                 "tool_descriptions": "\n".join(
#                     [f"    {tool.name}: {tool.description}" for tool in tools]
#                 )
#             },
#             template='\n'.join(
#                 [
#                     prefix,
#                     format_instructions,
#                     # context,
#                     question_prompt,
#                     suffix
#                 ]
#             )
#         )
#         human_message_prompt = HumanMessagePromptTemplate(prompt=human_prompt)
#         return ChatPromptTemplate.from_messages([human_message_prompt])

In [13]:
%%writefile utils/xml_prompt.py
from typing import Sequence
from langchain.tools import BaseTool
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate

AGENT_PREFIX = """You are an AI assistant. You always stays on topic of the human input and does not diverge from it."""

AGENT_FORMAT_INSTRUCTIONS = """
You have access to below tool descriptions:
<tool_descriptions>
{tool_descriptions}
</tool_descriptions>

The $XML_BLOB should only contain a SINGLE tool and MUST be formatted as markdown, do NOT return a list of multiple tools. Here is an example of a valid $XML_BLOB:
```xml
<tool>$TOOL_NAME</tool>
<tool_input>$INPUT</tool_input>
```
The tool in the $XML_BLOB should be one of [{tool_names}]
Make sure to have the $INPUT in the right format for the tool you are using, and do not put variable names as input if you can find the right values.

Use the following format:
Question: the input question you must answer.
Thought: you should always think about what to do
Action:
```xml
$XML_BLOB
```
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

You have access to the previous conversation below, where H refers to the human and A refers to the assistant:
<chat_history>
{chat_history}
</chat_history>
"""

AGENT_QUESTION_PROMPT = """Remember to respond with your knowledge when the question does not correspond to any tool. 
Always append the string "Final Answer:" when returning the final answer.

Question: {question}"""

AGENT_SUFFIX = """Thought: {agent_scratchpad}"""

def create_agent_prompt(
        tools: Sequence[BaseTool],
        prefix: str = AGENT_PREFIX,
        # context: str = AGENT_CONTEXT,
        format_instructions: str = AGENT_FORMAT_INSTRUCTIONS,
        question_prompt: str = AGENT_QUESTION_PROMPT,
        suffix: str = AGENT_SUFFIX,
    ) -> PromptTemplate:
    
        human_prompt = PromptTemplate(
            input_variables=[
                "chat_history", 
                "question",
                "agent_scratchpad"
            ],
            partial_variables={
                "tool_names": ", ".join([tool.name for tool in tools]),
                "tool_descriptions": "\n".join(
                    [f"    {tool.name}: {tool.description}" for tool in tools]
                )
            },
            template='\n'.join(
                [
                    prefix,
                    format_instructions,
                    # context,
                    question_prompt,
                    suffix
                ]
            )
        )
        human_message_prompt = HumanMessagePromptTemplate(prompt=human_prompt)
        return ChatPromptTemplate.from_messages([human_message_prompt])

Overwriting utils/xml_prompt.py


In [14]:
# import re, json

# pattern = re.compile(r"^.*?`{3}(?:xml)?\n?(.*?)`{3}.*?$", re.DOTALL)

# text = """Thought: you should always think about what to do
# Action: the tool in the $XML_BLOB should be one of [{tool_names}]
# ```xml
# <tool>$TOOL_NAME</tool>
# <tool_input>$INPUT</tool_input>
# ```
# Action Input: the input to the action
# Observation: the result of the action
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question"""

# found = pattern.search(text)
# match_object = found.group(1).strip()
# if "</tool>" in text:
#     tool, tool_input = text.split("</tool>")
#     _tool = tool.split("<tool>")[1]
#     _tool_input = tool_input.split("<tool_input>")[1]
#     if "</tool_input>" in _tool_input:
#         _tool_input = _tool_input.split("</tool_input>")[0]
#     print(f"tool: {_tool}")
#     print(f"too_input: {_tool_input}")
# else:
#     raise ValueError

In [15]:
import json
import re
from typing import Union

from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.exceptions import OutputParserException

from langchain.agents.agent import AgentOutputParser
from langchain.agents.chat.prompt import FORMAT_INSTRUCTIONS

FINAL_ANSWER_ACTION = "Final Answer:"

class ReActXMLSingleInputOutputParser(AgentOutputParser):
    """Parses ReAct-style LLM calls that have a single tool input in xml format.

    Expects output to be in one of two formats.

    If the output signals that an action should be taken,
    should be in the below format. This will result in an AgentAction
    being returned.

    ```
    Thought: agent thought here
    Action:
    ```xml
    <tool>$TOOL_NAME</tool>
    <tool_input>$INPUT</tool_input>
    ```

    If the output signals that a final answer should be given,
    should be in the below format. This will result in an AgentFinish
    being returned.

    ```
    Thought: agent thought here
    Final Answer: The temperature is 100 degrees
    ```

    """

    pattern = re.compile(r"^.*?`{3}(?:xml)?\n?(.*?)`{3}.*?$", re.DOTALL)
    """Regex pattern to parse the output."""

    def get_format_instructions(self) -> str:
        return FORMAT_INSTRUCTIONS

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        includes_answer = FINAL_ANSWER_ACTION in text
        try:
            found = self.pattern.search(text)
            if not found:
                # Fast fail to parse Final Answer.
                raise ValueError("action not found")
            action = found.group(1)
            response = action.strip()
            includes_tool = "</tool>" in response
            if includes_answer and includes_tool:
                raise OutputParserException(
                    "Parsing LLM output produced a final answer "
                    f"and a parse-able action: {text}"
                )
            elif includes_tool:
                tool, tool_input = text.split("</tool>")
                _tool = tool.split("<tool>")[1]
                _tool_input = tool_input.split("<tool_input>")[1]
                if "</tool_input>" in _tool_input:
                    _tool_input = _tool_input.split("</tool_input>")[0]
                return AgentAction(tool=_tool, tool_input=_tool_input, log=text)
        
        except Exception:
            if not includes_answer:
                raise OutputParserException(f"Could not parse LLM output: {text}")
            output = text.split(FINAL_ANSWER_ACTION)[-1].strip()
            return AgentFinish({"output": output}, text)
                

    # def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
    #     includes_answer = FINAL_ANSWER_ACTION in text
    #     if "</tool>" in text:
    #         tool, tool_input = text.split("</tool>")
    #         _tool = tool.split("<tool>")[1]
    #         _tool_input = tool_input.split("<tool_input>")[1]
    #         if "</tool_input>" in _tool_input:
    #             _tool_input = _tool_input.split("</tool_input>")[0]
    #         return AgentAction(tool=_tool, tool_input=_tool_input, log=text)
    #     elif "<final_answer>" in text:
    #         _, answer = text.split("<final_answer>")
    #         if "</final_answer>" in answer:
    #             answer = answer.split("</final_answer>")[0]
    #         return AgentFinish(return_values={"output": answer}, log=text)
    #     else:
    #         raise ValueError

    @property
    def _type(self) -> str:
        return "react-xml-single-input"


output_parser = ReActXMLSingleInputOutputParser()

### agents

In [32]:
from utils.xml_prompt import create_agent_prompt
from utils.tools import tools

# tools = tools[:2] 
# print(f"len(tools): {len(tools)}")

prompt = create_agent_prompt(tools)
print(prompt.messages[0].prompt.template)

llm = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    # model_id="anthropic.claude-3-haiku-20240307-v1:0",
    client=boto3.client("bedrock-runtime"),
    model_kwargs={
        "max_tokens": 4096, 
        "temperature": 0.5,
    },
    streaming=False,
    region_name="us-west-2",
    # region_name="us-east-1",
)

agent = (
    {
        "question": lambda x: x["question"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        # "agent_scratchpad": lambda x: x["intermediate_steps"],
        # "chat_history": lambda x: get_buffer_string(x["chat_history"])
        "chat_history": lambda x: x["chat_history"]
    }
    | prompt
    | llm.bind(stop=["\nObservation"])
    | output_parser
)

memory = ConversationBufferMemory(
    memory_key="chat_history", 
    return_messages=True, 
    input_key="question",
    output_key="output",
    ai_prefix="A",
    human_prefix="H",
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True
)

You are an AI assistant. You always stays on topic of the human input and does not diverge from it.

You have access to below tool descriptions:
<tool_descriptions>
{tool_descriptions}
</tool_descriptions>

The $XML_BLOB should only contain a SINGLE tool and MUST be formatted as markdown, do NOT return a list of multiple tools. Here is an example of a valid $XML_BLOB:
```xml
<tool>$TOOL_NAME</tool>
<tool_input>$INPUT</tool_input>
```
The tool in the $XML_BLOB should be one of [{tool_names}]
Make sure to have the $INPUT in the right format for the tool you are using, and do not put variable names as input if you can find the right values.

Use the following format:
Question: the input question you must answer.
Thought: you should always think about what to do
Action:
```xml
$XML_BLOB
```
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the f

### test

#### simple

In [10]:
%%time
result = agent_executor.invoke({
    "question": "what are Einstein's top 3 research papers ?",
})



> Entering new AgentExecutor chain...
To find Einstein's top 3 research papers, we can search on Google Scholar or arXiv which contain scientific publications.

```xml
<tool>search-google-scholar</tool>
<tool_input>Albert Einstein top research papers</tool_input>
```

Action Input: Albert Einstein top research papersTitle: Albert Einstein
Authors: 
Summary: A Einstein - Quantum Algebra and Symmetry, 2005 - researchgate.net
Total-Citations: 30

Title: The exceptional brain of Albert Einstein
Authors: 
Summary: SF Witelson, DL Kigar, T Harvey - The lancet, 1999 - thelancet.com
Total-Citations: 359

Title: Albert Einstein: Person of the century
Authors: 
Summary: F Golden - TIME-NEW YORK-, 1999 - content.time.com
Total-Citations: 36

Title: The corpus callosum of Albert Einstein's brain: another clue to his high intelligence?
Authors: D Falk,D Yin
Summary: W Men, D Falk, T Sun, W Chen, J Li, D Yin, L Zang… - Brain, 2014 - academic.oup.com
Total-Citations: 94

Title: Albert Einstein: cre

In [8]:
print(result["output"])

Einstein's top 3 most influential research papers are:
1. His 1905 paper on the Special Theory of Relativity
2. His 1905 paper on Brownian motion (which won the Nobel Prize)  
3. His 1905 paper on the photoelectric effect and quantum theory of light


In [21]:
print(result["output"])

Based on my research, Albert Einstein's three most influential and important scientific papers are:

1. His 1905 paper on the photoelectric effect, which laid the groundwork for the quantum theory of light and earned him the 1921 Nobel Prize in Physics. This paper proposed that light is made up of individual particles (photons) with discrete amounts of energy.

2. His 1905 paper introducing his special theory of relativity, which revolutionized our understanding of space, time, energy and matter. This paper derived the famous equation E=mc^2 showing the equivalence of mass and energy. 

3. His 1916 paper presenting his general theory of relativity, which described gravity not as a force, but as a consequence of the curvature of spacetime caused by the presence of mass and energy. This theory expanded the theory of relativity to include acceleration and gravity.

These three groundbreaking papers from Einstein's "Miracle Year" of 1905 and his subsequent work on general relativity in 191

In [9]:
%%time
result = agent_executor.invoke({
    "question": "show me the summary of the above first paper"
})



> Entering new AgentExecutor chain...
Thought: To get a summary of Einstein's 1905 paper on the Special Theory of Relativity, I should search for it on Wikipedia or use a scientific literature search tool like Google Scholar or arXiv.

Action: search-wikipedia
Action Input: Albert Einstein Special Theory of Relativity 1905 paper
Page: Theory of relativity
Summary: The theory of relativity usually encompasses two interrelated physics theories by Albert Einstein: special relativity and general relativity, proposed and published in 1905 and 1915, respectively. Special relativity applies to all physical phenomena in the absence of gravity. General relativity explains the law of gravitation and its relation to the forces of nature. It applies to the cosmological and astrophysical realm, including astronomy.The theory transformed theoretical physics and astronomy during the 20th century, superseding a 200-year-old theory of mechanics created primarily by Isaac Newton. It introduced concept

In [10]:
print(result["output"])

Einstein's 1905 paper "On the Electrodynamics of Moving Bodies" laid out the foundational principles of his special theory of relativity. The key points and findings in the paper include:

1. Introducing two fundamental postulates - the laws of physics are the same in all inertial (non-accelerating) frames of reference, and the speed of light in vacuum is independent of the motion of the light source. 

2. Deriving the counterintuitive consequences of these postulates, such as the relativity of simultaneity, length contraction, and time dilation effects for objects moving at relativistic speeds close to the speed of light.

3. Providing the mathematical formalism of Lorentz transformations to relate measurements of space and time coordinates between different inertial frames.

4. Showing that the idea of a luminiferous aether to propagate light waves is unnecessary, resolving difficulties with Maxwell's equations.

5. Establishing the equivalence of mass and energy through the famous e

#### next

In [25]:
%%time
result = agent_executor.invoke({
    "question": "what are Einstein's top 3 research papers ?",
})



> Entering new AgentExecutor chain...


Thought: To find Einstein's top 3 research papers, I should search a database of scientific publications like arXiv or Google Scholar.

Action:
```xml
<tool>search-google-scholar</tool>
<tool_input>einstein top research papers</tool_input>
```

Action Input: einstein top research papersTitle: Einstein metrics, spinning top motions and monopoles
Authors: 
Summary: H Pedersen - Mathematische Annalen, 1986 - Springer
Total-Citations: 117

Title: A stripe phase with supersolid properties in spin–orbit-coupled Bose–Einstein condensates
Authors: J Lee,S Burchesky,B Shteynas
Summary: …, J Lee, W Huang, S Burchesky, B Shteynas, FÇ Top… - Nature, 2017 - nature.com
Total-Citations: 520

Title: Output coupling of a Bose-Einstein condensate formed in a TOP trap
Authors: JL Martin
Summary: JL Martin, CR McKenzie, NR Thomas… - Journal of Physics B …, 1999 - iopscience.iop.org
Total-Citations: 57

Title: Observation of a spinning top in a Bose-Einstein condensate
Authors: RN Bisset,S Serafini,M Barbi

In [26]:
print(result["output"])

Albert Einstein's top 3 most influential research papers are:
1. His 1905 paper on the Special Theory of Relativity
2. His 1905 paper on the photoelectric effect, for which he won the Nobel Prize 
3. His 1905 paper providing experimental proof of atomic theory through explaining Brownian motion


In [27]:
%%time
result = agent_executor.invoke({
    "question": "show me the summary of the above first paper"
})



> Entering new AgentExecutor chain...
Thought: To get a summary of Einstein's 1905 paper on the Special Theory of Relativity, I should search for it on a scientific literature database like arXiv.
Action:
```xml
<tool>search-arxiv</tool>
<tool_input>einstein 1905 special theory of relativity</tool_input>
```
Action Input: einstein 1905 special theory of relativityPublished: 2012-04-30
Title: Poincaré's Dynamics of the Electron - A Theory of Relativity?
Authors: Galina Weinstein
Summary: Before 1905, Poincar\'e stressed the importance of the method of clocks and
their synchronization, but unlike Einstein, magnet and conductor (asymmetries
in Lorentz's theory regarding the explanation of Faraday's induction) or
chasing a light beam and overtaking it, were not a matter of great concern for
him. In 1905 Poincar\'e elaborated Lorentz's electron theory from 1904 in two
papers entitled "Sur la dynamique de l'electron". In May 1905 he sent three
letters to Lorentz at the same time that Alber

In [28]:
print(result["output"])

Here is a summary of Albert Einstein's 1905 paper "On the Electrodynamics of Moving Bodies" which introduced the special theory of relativity:

The paper laid out two fundamental principles - 1) The laws of physics are the same for all observers in relative motion in a vacuum, and 2) The speed of light in a vacuum is independent of the motion of its source. 

From these principles, Einstein derived that the laws of physics predict a lack of an absolute reference frame. Instead, the concepts of space and time are relative - lengths contract and time dilates for moving observers, in a way defined by the mathematical Lorentz transformations.

The paper showed that the long-held concepts of absolute space and time from Newtonian physics were invalid, and that mass and energy are equivalent and related by the famous equation E=mc^2. This revolutionary work by Einstein completely upended the accepted framework of physics at the time.


### testing for reflexion

In [33]:
%%time
result = agent_executor.invoke({
    # "question": "What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into? Search for Colorado orogeny, High Plains from United States in this order."
    "question": "What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?"
})



> Entering new AgentExecutor chain...


Thought: To answer this question about the elevation range in the area affected by the eastern sector of the Colorado orogeny, I will need to search for information on the geology and geography of that region.

Action:
```xml
<tool>search-wikipedia</tool>
<tool_input>Colorado orogeny eastern sector</tool_input>
```

Action Input: Colorado orogeny eastern sectorPage: Colorado orogeny
Summary: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas. This took place from 1780 to 1650 million years ago (Mya), during the Paleoproterozoic (Statherian Period). It is recorded in the Colorado orogen, a >500-km-wide belt of oceanic arc rock that extends southward into New Mexico. The Colorado orogeny was likely part of the larger Yavapai orogeny.The Wikipedia search provided some background information on the Colorado orogeny, but did not directly answer the question about the elevation range in the area affected by the eastern sector. To better an

In [42]:
%%time
result = agent_executor.invoke({
    "question": "Which character does this protagonist, who secretly loves and marries a member of the rival house, of William Shakespeare's tragedy that has a fictional character Benvolio slay?"
})



> Entering new AgentExecutor chain...


Thought: This question is asking about a character from a William Shakespeare tragedy that involves two rival houses, a secret marriage between members of those houses, and a character named Benvolio who slays someone. Based on those details, it seems to be referring to the play Romeo and Juliet. To confirm and provide the specific character name, I should search Wikipedia for information on that play.

Action:
```xml
<tool>search-wikipedia</tool>
<tool_input>Romeo and Juliet play</tool_input>
```

Action Input: Romeo and Juliet playPage: Romeo and Juliet
Summary: Romeo and Juliet is a tragedy written by William Shakespeare early in his career about the romance between two Italian youths from feuding families. It was among Shakespeare's most popular plays during his lifetime and, along with Hamlet, is one of his most frequently performed. Today, the title characters are regarded as archetypal young lovers.
Romeo and Juliet belongs to a tradition of tragic romances stretching back to an

In [47]:
question = "Alice David is the voice of Lara Croft in a video game developed by which company ?"
result = agent_executor.invoke({
    "question": question
})




> Entering new AgentExecutor chain...


Thought: To answer this question, I will need to search for information about the video game series featuring the character Lara Croft and the company that developed those games.

Action:
```xml
<tool>search-wikipedia</tool>
<tool_input>Lara Croft video game series</tool_input>
```

Action Input: Lara Croft video game seriesPage: Lara Croft
Summary: Lara Croft is a character and the main protagonist of the video game franchise Tomb Raider. She is presented as a highly intelligent and athletic British archaeologist who ventures into ancient tombs and hazardous ruins around the world. Created by a team at British developer Core Design that included Toby Gard, the character first appeared in the video game Tomb Raider in 1996.
Core Design handled the initial development of the character and the series. Inspired by strong female icons, Gard designed Lara Croft to counter stereotypical female characters. The company modified the character for subsequent titles, which included graphical impr

In [48]:
print(result["output"])

The company that developed the video game series where Alice David voiced Lara Croft is Core Design.


### references

try modifying prompts for better performance. take inspiration from:
- https://github.com/langchain-ai/langchain/issues/18434
- https://smith.langchain.com/hub/shoggoth13/xml-agent?organizationId=00a1fc16-cd40-5852-bcf2-9353e12f11ee